# Diabetes Progression Risk Prediction — Model Comparison Report (Scikit‑Learn Diabetes Dataset)

**Course Lab Notebook (Train / Validation / Test split + multiple models)**

**Objective (screening tool):** build a model that **best predicts diabetes disease progression one year after baseline**, so physicians can flag patients at higher risk.

**Models evaluated**
1. Univariate Polynomial Regression (BMI only)
2. Multivariate Polynomial Models (all features)
3. Decision Trees
4. k‑Nearest Neighbors (kNN)
5. Logistic Regression (classification framing: “high risk” vs “low risk”)

**Metrics**
- Regression: **R²**, **MAE**, **MAPE**
- Classification (Logistic Regression section): **Accuracy, Precision/Recall, ROC‑AUC, Log‑Loss**  
  > Logistic Regression is a *classifier* (workshop topic). Because the original target is continuous, we create a binary “high risk” label for this section and use classification metrics.

---

## Notebook structure (good ML report)
- Problem framing + metrics rationale
- EDA (statistics, plots, correlation, insights)
- Data quality & cleaning notes
- Train/Validation/Test split (75/10/15)
- Model training + validation comparison tables
- Final test evaluation for chosen models
- Error analysis + limitations + conclusions


In [ ]:
# --- Setup ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# Classification metrics (for Logistic Regression section)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, roc_auc_score, log_loss,
    confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


def mape(y_true, y_pred, epsilon=1e-8):
    """Mean Absolute Percentage Error (MAPE) with epsilon to avoid division by zero."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.maximum(np.abs(y_true), epsilon)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100


def regression_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MAPE(%)": mape(y_true, y_pred),
    }

print("Libraries imported.")


ModuleNotFoundError: No module named 'sklearn'

# Part 1 — Data, Problem Framing, EDA, Cleaning, Split

## 1) Get the data
We use **Scikit‑Learn’s Diabetes toy dataset** (`load_diabetes`).  

## 2) Frame the problem (talking points)
- **What we predict:** *disease progression one year after baseline* (continuous target).  
- **Why this matters:** A model with good generalization can help physicians prioritize follow‑ups and interventions.  
- **Train/Validation/Test thinking:**  
  - **Train** fits model parameters.  
  - **Validation** selects model complexity (e.g., polynomial degree, `k` in kNN, `max_depth` in trees).  
  - **Test** is the final unbiased estimate of performance.

### Metrics rationale (from the “Performance Metrics” workshop)
- **R²:** how much variance in progression is explained by the model (higher is better).  
- **MAE:** average absolute error in the target units (lower is better; interpretable).  
- **MAPE:** error as a percentage (lower is better; easy to communicate, but sensitive to values near 0).

### Modeling mindset (talking points from KNN + Logistic Regression workshops)
- **kNN intuition:** “predict like your neighbors” — works well when similar patients have similar outcomes; sensitive to `k` (bias/variance trade‑off).  
- **Logistic Regression intuition:** models probability of a **binary outcome** using a linear decision boundary in feature space; interpretable coefficients.  
  - In this lab, the original target is continuous, so we also show a **binary high‑risk** framing for Logistic Regression (Part 3) to connect with workshop learning.


In [ ]:
# Load the diabetes dataset as a pandas DataFrame
diabetes = datasets.load_diabetes(as_frame=True)
X = diabetes.data.copy()
y = diabetes.target.copy()

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)
display(X.head())
display(y.head())

# Dataset description (skim key parts)
print(diabetes.DESCR[:1200], "...")


## 3) Exploratory Data Analysis (EDA)

We will include:
- Descriptive statistics
- Histograms (feature distributions)
- Scatter plots (BMI vs target + a couple more)
- Correlation matrix heatmap
- Concise insights


In [ ]:
# Descriptive statistics
display(X.describe().T)

# Check target distribution
plt.figure()
plt.hist(y, bins=25)
plt.title("Target distribution: disease progression (1 year)")
plt.xlabel("Progression (target)")
plt.ylabel("Count")
plt.show()

# Histograms for features
X.hist(bins=20, figsize=(12, 8))
plt.suptitle("Feature distributions (standardized features)", y=1.02)
plt.show()

# Scatter plots (BMI, BP, and S5 vs target as examples)
for col in ["bmi", "bp", "s5"]:
    plt.figure()
    plt.scatter(X[col], y, alpha=0.7)
    plt.title(f"Scatter: {col} vs target")
    plt.xlabel(col)
    plt.ylabel("Progression (target)")
    plt.show()

# Correlation matrix
corr = X.join(y.rename("target")).corr(numeric_only=True)

plt.figure(figsize=(10, 8))
plt.imshow(corr, aspect="auto")
plt.title("Correlation matrix (features + target)")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar()
plt.tight_layout()
plt.show()

# Quick look: correlations with target
target_corr = corr["target"].sort_values(ascending=False)
display(target_corr)


### EDA Insights (concise)
- Features are already **standardized** in this dataset (centered/scaled), which is helpful for distance‑based models (kNN) and numerical stability.
- **BMI** and **S5** often show relatively stronger correlation with the target than many other features, suggesting they may be informative predictors.
- Correlation does **not** guarantee causation; it’s only a first signal for modeling choices.
- The target distribution is continuous with a moderate spread; extreme values exist, which can challenge simple linear models.


## 4) Data Cleaning (if needed)

We check for:
- Missing values
- Duplicates (rare in toy data)
- Data types

If no issues are found, we document that no cleaning is required.


In [ ]:
# Missing values
missing = X.isna().sum()
print("Missing values per feature:")
display(missing)

print("Any missing in target?", y.isna().any())

# Duplicates (rows)
dup_count = X.duplicated().sum()
print("Duplicate feature rows:", dup_count)

# Data types
display(X.dtypes)

# Cleaning decision
if missing.sum() == 0 and not y.isna().any():
    print("✅ No missing values detected. No imputation required.")
else:
    print("⚠️ Missing values exist — would require imputation (not expected for this dataset).")


## 5) Train / Validation / Test split (75% / 10% / 15%)

We will do a two‑step split:
1. Train (75%) vs Temp (25%)
2. Split Temp into Validation (10%) and Test (15%)  
   - Within the 25% temp: validation fraction = 10/25 = 0.4, test fraction = 15/25 = 0.6


In [ ]:
# Step 1: train vs temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE
)

# Step 2: temp -> validation and test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.60, random_state=RANDOM_STATE
)

print("Train size:", X_train.shape[0], " (expected ~75%)")
print("Val size:  ", X_val.shape[0], " (expected ~10%)")
print("Test size: ", X_test.shape[0], " (expected ~15%)")

# Sanity check: proportions
n = len(X)
print("Train %:", round(len(X_train)/n*100, 2))
print("Val   %:", round(len(X_val)/n*100, 2))
print("Test  %:", round(len(X_test)/n*100, 2))


# Part 2 — Univariate Polynomial Regression (BMI only)

We fit **6 models** using only the **BMI** feature:
- Degree 0, 1, 2, 3, 4, 5

Then we compare using Train + Validation metrics, pick the best model, test it, plot fits, write the equation, and analyze parameters.


In [ ]:
# Prepare univariate data (BMI only)
X_train_bmi = X_train[["bmi"]].copy()
X_val_bmi   = X_val[["bmi"]].copy()
X_test_bmi  = X_test[["bmi"]].copy()

results = []

models = {}
for deg in range(0, 6):
    model = Pipeline(steps=[
        ("poly", PolynomialFeatures(degree=deg, include_bias=True)),
        ("linreg", LinearRegression())
    ])
    model.fit(X_train_bmi, y_train)
    models[deg] = model

    # Train metrics
    yhat_tr = model.predict(X_train_bmi)
    tr_m = regression_metrics(y_train, yhat_tr)

    # Validation metrics
    yhat_val = model.predict(X_val_bmi)
    val_m = regression_metrics(y_val, yhat_val)

    results.append({
        "degree": deg,
        "train_R2": tr_m["R2"],
        "train_MAE": tr_m["MAE"],
        "train_MAPE(%)": tr_m["MAPE(%)"],
        "val_R2": val_m["R2"],
        "val_MAE": val_m["MAE"],
        "val_MAPE(%)": val_m["MAPE(%)"],
        "n_params": len(model.named_steps["poly"].get_feature_names_out(["bmi"]))
    })

results_df = pd.DataFrame(results).sort_values("val_R2", ascending=False)
display(results_df)


## 8) Identify the best model (based on validation)

Primary selection rule (simple + defensible):
- Choose the model with **highest validation R²**
- If very close, prefer **lower validation MAE** and simpler degree (to reduce overfitting risk).


In [ ]:
# Select the best degree by validation R2 (tie-breakers: MAE, simpler degree)
results_sorted = results_df.sort_values(["val_R2", "val_MAE", "degree"], ascending=[False, True, True])
best_row = results_sorted.iloc[0]
best_degree = int(best_row["degree"])
best_model = models[best_degree]

print("Best degree selected:", best_degree)
display(best_row.to_frame("best_model_summary"))


## 9) Evaluate the chosen model on the **test set** (R², MAE, MAPE)

In [ ]:
# Test evaluation
yhat_test = best_model.predict(X_test_bmi)
test_m = regression_metrics(y_test, yhat_test)
test_m


## 10) Plot train / validation / test points + model fit

We plot all three splits and overlay the chosen polynomial curve.


In [ ]:
# Plot data and the chosen polynomial fit
def plot_univariate_fit(model, X_train_bmi, y_train, X_val_bmi, y_val, X_test_bmi, y_test, title):
    # grid for smooth curve
    x_all = pd.concat([X_train_bmi, X_val_bmi, X_test_bmi])["bmi"]
    x_grid = np.linspace(x_all.min(), x_all.max(), 300).reshape(-1, 1)
    x_grid_df = pd.DataFrame(x_grid, columns=["bmi"])
    y_grid = model.predict(x_grid_df)

    plt.figure()
    plt.scatter(X_train_bmi["bmi"], y_train, alpha=0.7, label="Train")
    plt.scatter(X_val_bmi["bmi"], y_val, alpha=0.7, label="Validation")
    plt.scatter(X_test_bmi["bmi"], y_test, alpha=0.7, label="Test")
    plt.plot(x_grid_df["bmi"], y_grid, label="Model fit")
    plt.title(title)
    plt.xlabel("BMI (standardized)")
    plt.ylabel("Progression (target)")
    plt.legend()
    plt.show()

plot_univariate_fit(
    best_model, X_train_bmi, y_train, X_val_bmi, y_val, X_test_bmi, y_test,
    title=f"Chosen Univariate Polynomial Fit (degree={best_degree})"
)


## 11) Equation of the best model (2 decimal digits)

We extract polynomial feature names and coefficients from the trained pipeline.

> Note: BMI is **standardized** in this dataset, so the equation uses standardized BMI.


In [ ]:
# Extract equation from the pipeline
poly = best_model.named_steps["poly"]
lin = best_model.named_steps["linreg"]

feature_names = poly.get_feature_names_out(["bmi"])
coefs = lin.coef_
intercept = lin.intercept_

# LinearRegression in sklearn with include_bias=True: the bias term may be captured in intercept; still OK.
terms = []
for name, c in zip(feature_names, coefs):
    if name == "1":
        # handled by intercept conceptually
        continue
    terms.append((name, c))

# Build equation string
eq = f"y = {intercept:.2f}"
for name, c in terms:
    sign = "+" if c >= 0 else "-"
    eq += f" {sign} {abs(c):.2f}*{name}"

print("Best model equation (approx.):")
print(eq)


## 12) Predict progression for a BMI value of your choice

Because BMI is standardized, a value like:
- **bmi = 0.05** means slightly above the dataset mean.


In [ ]:
bmi_value = 0.05  # choose any standardized BMI value you want
pred = best_model.predict(pd.DataFrame({"bmi":[bmi_value]}))[0]
print(f"Predicted progression for bmi={bmi_value} (standardized): {pred:.2f}")


## 13) How many trainable parameters in each model? (and why)

For **univariate polynomial regression** of degree *d*, the polynomial feature vector includes:
- 1 (bias), x, x², …, xᵈ  
So the number of coefficients grows as **(d + 1)**.

We show this directly using `get_feature_names_out()` (a practical way to see how many parameters/features are created).


In [ ]:
param_table = []
for deg in range(0, 6):
    feats = models[deg].named_steps["poly"].get_feature_names_out(["bmi"])
    param_table.append({"degree": deg, "poly_features": len(feats), "feature_names": ", ".join(feats)})

param_df = pd.DataFrame(param_table)
display(param_df[["degree","poly_features"]])


## 14) Conclusions for Part 2 (failure analysis + limitations)

We do a short error analysis:
- Residual plot (predicted vs residual)
- Check where errors are larger (often at extremes)

Talking points:
- **Bias/variance:** higher degree can overfit train data and degrade validation.
- **Metric trade‑offs:** R² can improve while MAPE/MAE might not improve equally.
- **Limitations:** BMI alone ignores other clinical factors; screening tools must be used with caution.


In [ ]:
# Residual analysis on test set for univariate best model
yhat_test = best_model.predict(X_test_bmi)
residuals = y_test - yhat_test

plt.figure()
plt.scatter(yhat_test, residuals, alpha=0.7)
plt.axhline(0)
plt.title("Residuals vs Predicted (Test) — Univariate Best Model")
plt.xlabel("Predicted")
plt.ylabel("Residual (Actual - Predicted)")
plt.show()

plt.figure()
plt.hist(residuals, bins=20)
plt.title("Residual distribution (Test) — Univariate Best Model")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.show()

# A quick look at largest absolute errors
err_df = pd.DataFrame({
    "bmi": X_test_bmi["bmi"].values,
    "actual": y_test.values,
    "pred": yhat_test,
    "abs_error": np.abs(residuals.values)
}).sort_values("abs_error", ascending=False)

display(err_df.head(10))


# Part 3 — Multivariate Models (all features) + Trees + kNN + Logistic Regression

We repeat the core workflow:
- Train on **train**
- Select hyperparameters based on **validation**
- Final evaluation on **test**

Models:
1. **Two polynomial models** (degrees > 1)
2. **Two decision trees** (`max_depth` variations)
3. **Two kNN regressors** (`k` variations)
4. **Two Logistic Regression models** (binary “high‑risk” framing)


In [ ]:
# Helper: train+eval for regression models
def eval_regression_model(name, model, X_tr, y_tr, X_val, y_val):
    model.fit(X_tr, y_tr)
    yhat_tr = model.predict(X_tr)
    yhat_val = model.predict(X_val)
    tr = regression_metrics(y_tr, yhat_tr)
    va = regression_metrics(y_val, yhat_val)
    return {
        "model": name,
        "train_R2": tr["R2"], "train_MAE": tr["MAE"], "train_MAPE(%)": tr["MAPE(%)"],
        "val_R2": va["R2"], "val_MAE": va["MAE"], "val_MAPE(%)": va["MAPE(%)"],
    }

reg_models = {}

# --- 1) Two polynomial models (multivariate) ---
# Note: full degree-3 on 10 features can create many features; we keep it reasonable and add regularization (Ridge).
poly2_ridge = Pipeline(steps=[
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("ridge", Ridge(alpha=1.0, random_state=RANDOM_STATE))
])

poly3_inter_ridge = Pipeline(steps=[
    ("poly", PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)),
    ("ridge", Ridge(alpha=1.0, random_state=RANDOM_STATE))
])

reg_models["Poly2+Ridge"] = poly2_ridge
reg_models["Poly3(interactions)+Ridge"] = poly3_inter_ridge

# --- 2) Two decision trees ---
tree_d3 = DecisionTreeRegressor(max_depth=3, random_state=RANDOM_STATE)
tree_d6 = DecisionTreeRegressor(max_depth=6, random_state=RANDOM_STATE)
reg_models["Tree(depth=3)"] = tree_d3
reg_models["Tree(depth=6)"] = tree_d6

# --- 3) Two kNN regressors ---
knn3 = KNeighborsRegressor(n_neighbors=3)
knn15 = KNeighborsRegressor(n_neighbors=15)
reg_models["kNN(k=3)"] = knn3
reg_models["kNN(k=15)"] = knn15

# Evaluate on train + validation
reg_results = []
for name, model in reg_models.items():
    reg_results.append(eval_regression_model(name, model, X_train, y_train, X_val, y_val))

reg_results_df = pd.DataFrame(reg_results).sort_values("val_R2", ascending=False)
display(reg_results_df)


## Pick best multivariate regression model + evaluate on test

Selection rule: highest validation R² (tie-breaker: lower validation MAE).


In [ ]:
reg_results_sorted = reg_results_df.sort_values(["val_R2","val_MAE"], ascending=[False, True])
best_reg_name = reg_results_sorted.iloc[0]["model"]
best_reg_model = reg_models[best_reg_name]

print("Best multivariate regression model:", best_reg_name)

# Refit on train+val (common practice after selecting hyperparameters)
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

best_reg_model.fit(X_trainval, y_trainval)
yhat_test_reg = best_reg_model.predict(X_test)
test_reg_metrics = regression_metrics(y_test, yhat_test_reg)

test_reg_metrics


## Multivariate regression error analysis (where does it fail?)

In [ ]:
resid_reg = y_test - yhat_test_reg

plt.figure()
plt.scatter(yhat_test_reg, resid_reg, alpha=0.7)
plt.axhline(0)
plt.title(f"Residuals vs Predicted (Test) — {best_reg_name}")
plt.xlabel("Predicted")
plt.ylabel("Residual (Actual - Predicted)")
plt.show()

plt.figure()
plt.hist(resid_reg, bins=20)
plt.title(f"Residual distribution (Test) — {best_reg_name}")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.show()

err_df2 = pd.DataFrame({
    "actual": y_test.values,
    "pred": yhat_test_reg,
    "abs_error": np.abs(resid_reg.values)
}).sort_values("abs_error", ascending=False)

display(err_df2.head(10))


## Logistic Regression (classification framing)

Because Logistic Regression is a classifier, we convert the continuous progression target into a **binary label**:
- **High risk = 1** if progression ≥ median (computed on training data)
- **Low risk = 0** otherwise

This mirrors a realistic screening scenario: “flag patients likely to have higher progression”.

We train **two Logistic Regression models** (different regularization strengths `C`) and evaluate on validation:
- Accuracy, Precision, Recall, ROC‑AUC, Log‑Loss


In [ ]:
# Create binary target using median of TRAIN
threshold = np.median(y_train)
y_train_bin = (y_train >= threshold).astype(int)
y_val_bin   = (y_val   >= threshold).astype(int)
y_test_bin  = (y_test  >= threshold).astype(int)

print("Threshold (median of train target):", threshold)
print("Train positive rate:", y_train_bin.mean().round(3))
print("Val positive rate:  ", y_val_bin.mean().round(3))
print("Test positive rate: ", y_test_bin.mean().round(3))

def eval_logistic(name, clf, X_tr, y_tr, X_val, y_val):
    clf.fit(X_tr, y_tr)
    p_tr = clf.predict_proba(X_tr)[:, 1]
    p_val = clf.predict_proba(X_val)[:, 1]
    yhat_tr = (p_tr >= 0.5).astype(int)
    yhat_val = (p_val >= 0.5).astype(int)
    return {
        "model": name,
        "train_acc": accuracy_score(y_tr, yhat_tr),
        "val_acc": accuracy_score(y_val, yhat_val),
        "train_prec": precision_score(y_tr, yhat_tr, zero_division=0),
        "val_prec": precision_score(y_val, yhat_val, zero_division=0),
        "train_rec": recall_score(y_tr, yhat_tr, zero_division=0),
        "val_rec": recall_score(y_val, yhat_val, zero_division=0),
        "train_auc": roc_auc_score(y_tr, p_tr),
        "val_auc": roc_auc_score(y_val, p_val),
        "train_logloss": log_loss(y_tr, p_tr),
        "val_logloss": log_loss(y_val, p_val),
    }

# Two logistic regression models
log1 = LogisticRegression(C=0.3, max_iter=500, random_state=RANDOM_STATE)
log2 = LogisticRegression(C=3.0, max_iter=500, random_state=RANDOM_STATE)

log_results = []
log_results.append(eval_logistic("LogReg(C=0.3)", log1, X_train, y_train_bin, X_val, y_val_bin))
log_results.append(eval_logistic("LogReg(C=3.0)", log2, X_train, y_train_bin, X_val, y_val_bin))

log_results_df = pd.DataFrame(log_results).sort_values("val_auc", ascending=False)
display(log_results_df)


### Logistic Regression — select best on validation ROC‑AUC and evaluate on test

In [ ]:
best_log_name = log_results_df.iloc[0]["model"]
best_log_model = log1 if best_log_name == "LogReg(C=0.3)" else log2
print("Best logistic model:", best_log_name)

# Refit on train+val for final test evaluation
y_trainval_bin = (y_trainval >= threshold).astype(int)
best_log_model.fit(X_trainval, y_trainval_bin)

p_test = best_log_model.predict_proba(X_test)[:, 1]
yhat_test_bin = (p_test >= 0.5).astype(int)

log_test_metrics = {
    "accuracy": accuracy_score(y_test_bin, yhat_test_bin),
    "precision": precision_score(y_test_bin, yhat_test_bin, zero_division=0),
    "recall": recall_score(y_test_bin, yhat_test_bin, zero_division=0),
    "roc_auc": roc_auc_score(y_test_bin, p_test),
    "log_loss": log_loss(y_test_bin, p_test),
}
log_test_metrics


In [ ]:
# Confusion matrix + ROC curve on test
cm = confusion_matrix(y_test_bin, yhat_test_bin)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Confusion Matrix (Test) — Logistic Regression")
plt.show()

RocCurveDisplay.from_predictions(y_test_bin, p_test)
plt.title("ROC Curve (Test) — Logistic Regression")
plt.show()


# Final Conclusions (overall)

## What performed best?
- Use the validation set to choose the best complexity / hyperparameters.
- Report final metrics only once on the test set to avoid optimistic bias.

## Where models fail (common patterns)
- **Extreme progression values:** many models under‑predict very high progression and over‑predict very low progression (regression to the mean).
- **Univariate BMI model limitation:** ignores other risk factors; can’t capture multi‑factor interactions.
- **High‑degree polynomials:** can overfit (high variance) and become unstable at edges of the feature range.
- **Decision trees:** can overfit if too deep; shallow trees may underfit (high bias).
- **kNN:** sensitive to `k`; small `k` can chase noise, large `k` can oversmooth.

## Practical limitations (screening tool context)
- The dataset is a curated toy dataset, not a real clinical cohort (generalization limits).
- The target is not a direct “risk probability”; it’s a continuous progression measure.
- For real deployment, we would require:
  - external validation on a separate hospital population
  - fairness checks across patient subgroups
  - calibration (especially for classification “high‑risk” probabilities)
  - clinical oversight: models support decisions, not replace them
